In [ ]:
import numpy as np
import pandas as pd

from bikewheelcalc import BicycleWheel, Rim, Hub, Spoke
import bikewheellib as bl
from doetools import LateralStiffnessDOE
from shutil import copy2
from imp import find_module

import matplotlib.pyplot as plt
from IPython.display import display

%matplotlib inline

In [ ]:
# Create ABAQUS simulations
R = 0.300
GJ = 25.0
EI1 = 100.
EI2 = GJ / 0.2

out_dir = '../data/doe/doe_Klat_asymm'

doe = LateralStiffnessDOE(out_dir=out_dir,
                          opts={'spk_paired': False, 'spk_eltype': 'beam'})

rim_offset = np.linspace(0., 0.024, 10)
tension = np.linspace(0., 1500., 10)
for r in rim_offset:

    wh = BicycleWheel()
    wh.hub = Hub(diam1=0.05, width1=0.025-r, width2=0.025+r)
    wh.rim = Rim(radius=R, area=100e-6,
                 I11=GJ / 26.0e9, I22=EI2 / 26.0e9, I33=EI1 / 26.0e9, Iw=0.0 / 69.0e9,
                 young_mod=69.0e9, shear_mod=26.0e9)

    wh.lace_radial(n_spokes=36, diameter=1.8e-3, young_mod=200e9, offset=0.00)

    for t in tension:
        jobname = 'Tc_dish_{:.3f}_T{:.0f}'.format(r, t)
        doe.add_experiment(wh, opts={'jobname': jobname, 'rim_offset': r, 'spk_T': t})

print('\nCreated {0:d} simulations'.format(len(doe.db)))

doe.write_input_files(N_batches=4)
doe.to_csv()

# Copy postprocessing script
copy2(src=find_module('doetools')[1] + '/postproc_lat_stiffness.py',
      dst=out_dir)

In [ ]:
# Load DOE database
print 'Loading database...'
doe = LateralStiffnessDOE(out_dir=out_dir, db_file=out_dir+'/_doe_db.csv')

# Populate DOE database with results
print 'Extracting results...'
d = doe.extract_results()
print '\nDone'

In [ ]:
def rr_lateral_stiffness(r, smeared_spokes=True, buckling=True, coupling=True):
    w = doe.wheel_from_row(r)
    
    w.apply_tension(r['spk_T'] + 0.1)

    rr = bl.ModeMatrix(w, N=36)
    
    return rr.calc_lat_stiff(smeared_spokes=smeared_spokes,
                             buckling=buckling,
                             coupling=coupling)

def rr_radial_stiffness(r, smeared_spokes=True, buckling=True, coupling=True):
    w = doe.wheel_from_row(r)
    
    w.apply_tension(r['spk_T'] + 0.1)

    rr = bl.ModeMatrix(w, N=36)
    
    return rr.calc_rad_stiff(smeared_spokes=smeared_spokes,
                             buckling=buckling,
                             coupling=coupling)

if True:
    doe.db['K_lat_cp_sm'] = doe.db.apply(lambda x: rr_lateral_stiffness(x, smeared_spokes=True, coupling=True), axis=1)
    doe.db['K_lat_cp_ds'] = doe.db.apply(lambda x: rr_lateral_stiffness(x, smeared_spokes=False, coupling=True), axis=1)
    doe.db['K_lat_nc_sm'] = doe.db.apply(lambda x: rr_lateral_stiffness(x, smeared_spokes=True, coupling=False), axis=1)
    
    doe.db['K_rad_cp_sm'] = doe.db.apply(lambda x: rr_radial_stiffness(x, smeared_spokes=True, coupling=True), axis=1)
    doe.db['K_rad_cp_ds'] = doe.db.apply(lambda x: rr_radial_stiffness(x, smeared_spokes=False, coupling=True), axis=1)
    doe.db['K_rad_nc_sm'] = doe.db.apply(lambda x: rr_radial_stiffness(x, smeared_spokes=True, coupling=False), axis=1)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(5., 2.5))


ax[0].plot(1000*doe.db[doe.db['spk_T'] == 0]['rim_offset'],
           doe.db[doe.db['spk_T'] == 0]['K_lat_nc_sm']/1000,
           'C0--', label=r'NC, SM')

ax[0].plot(1000*doe.db[doe.db['spk_T'] == 0]['rim_offset'],
           doe.db[doe.db['spk_T'] == 0]['K_lat_cp_sm']/1000,
           'C1', label=r'CP, SM')

ax[0].plot(1000*doe.db[doe.db['spk_T'] == 0]['rim_offset'],
           doe.db[doe.db['spk_T'] == 0]['K_lat_cp_ds']/1000,
           'C2s', label=r'CP, DS')

ax[0].plot(1000*doe.db[doe.db['spk_T'] == 0]['rim_offset'],
           doe.db[doe.db['spk_T'] == 0]['K_lat_abq']/1000,
           'C3*', label='ABAQUS')

ax[0].set_ylim([0., 180])
ax[0].set_xlim([0., 25.])
ax[0].set_xticks(range(0, 26, 5))

ax[0].set_xlabel('Rim offset [mm]')
ax[0].set_ylabel('Lateral stiffness [N/mm]')


ax[1].plot(1000*doe.db[doe.db['spk_T'] == 0]['rim_offset'],
           doe.db[doe.db['spk_T'] == 0]['K_rad_nc_sm']/1e6,
           'C0--', label='NC, SM')

ax[1].plot(1000*doe.db[doe.db['spk_T'] == 0]['rim_offset'],
           doe.db[doe.db['spk_T'] == 0]['K_rad_cp_sm']/1e6,
           'C1', label='CP, SM')

ax[1].plot(1000*doe.db[doe.db['spk_T'] == 0]['rim_offset'],
           doe.db[doe.db['spk_T'] == 0]['K_rad_cp_ds']/1e6,
           'C2s', label='CP, DS')

ax[1].set_ylim([0., 7.])
ax[1].set_xlim([0., 25.])
ax[1].set_xticks(range(0, 26, 5))

ax[1].set_xlabel('Rim offset [mm]')
ax[1].set_ylabel('Radial stiffness [kN/mm]')

ax[0].legend()

plt.tight_layout()
plt.savefig('../figs/stress_analysis/_python_asymm_Klat.pdf')